In [53]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv('taxi.csv')
data[['pickup_datetime', 'dropoff_datetime']] = data[['pickup_datetime', 'dropoff_datetime']].apply(pd.to_datetime)

In [54]:
data['weekday'] = data['pickup_datetime'].dt.weekday
data

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,weekday
0,id2863541,2,2016-01-31 13:12:20,2016-01-31 13:21:37,2,-73.989609,40.741718,-73.991013,40.755867,N,557,6
1,id1752141,1,2016-05-03 18:20:59,2016-05-03 18:41:09,1,-73.993935,40.761517,-73.975281,40.751507,N,1210,1
2,id1331142,1,2016-06-05 02:20:39,2016-06-05 02:28:04,1,-73.988625,40.722538,-73.980347,40.742752,N,445,6
3,id0227018,2,2016-01-11 20:07:13,2016-01-11 20:14:26,1,-73.976723,40.775311,-73.964333,40.765560,N,433,0
4,id2263790,2,2016-01-22 00:31:59,2016-01-22 00:40:48,5,-74.005852,40.740360,-73.988708,40.758781,N,529,4
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,id1085998,2,2016-05-10 17:09:15,2016-05-10 17:28:13,1,-73.990089,40.751648,-73.981262,40.784264,N,1138,1
99996,id1119642,1,2016-02-13 23:25:04,2016-02-13 23:32:14,1,-73.985161,40.774433,-73.976547,40.790916,N,430,5
99997,id3739873,2,2016-04-19 17:22:31,2016-04-19 17:43:12,1,-73.975052,40.756241,-73.984818,40.769341,N,1241,1
99998,id1863124,2,2016-06-22 20:50:36,2016-06-22 21:04:30,1,-73.983192,40.744656,-73.950607,40.783321,N,834,2


In [55]:
filtered_data = data[data['weekday'] >= 5]

In [56]:
filtered_data['month'] = filtered_data['dropoff_datetime'].dt.month
filtered_data[['id', 'month']].groupby(by='month').aggregate('count')

,id
month,
1,4609
2,4651
3,4694
4,5275
5,4981
6,4201


In [57]:
# ### как я рассчитываю день недели https://lifehacker.ru/kakoj-den-nedeli/
# ### столетия учитывать излишне
# def check_leap_year(year: int) -> bool:
#     return (year % 4 == 0) and (year % 100 != 0 or year % 400 == 0)


# def get_code_of_year(year: int) -> int:
#     last_two_digits = int(year % 100)
#     return (6 + last_two_digits + int(last_two_digits / 4)) % 7


# def get_code_of_month(month: int, year: int) -> int:
#     assert month in range(1, 12 + 1), f'Номер месяца в году может быть от 1 до 12, а на входе месяц {month}'
#     months_codes = {
#         1: 0 if check_leap_year(year) else 1,
#         2: 3 if check_leap_year(year) else 4,
#         3: 4, 
#         4: 0,
#         5: 2,
#         6:5,
#         7: 0,
#         8: 3,
#         9: 6,
#         10: 1,
#         11: 4,
#         12: 6
#     }
#     return months_codes[month]


# def get_day_of_week(day: int, month: int, year: int) -> int:
#     assert day in range(1, 31 + 1), f'Номер дня в месяце должен быть от 1 до 31 включительно, а на входе день {day}'
#     days_of_week = {
#         0: 6,
#         1: 7,
#         2: 1,
#         3: 2,
#         4: 3,
#         5: 4,
#         6: 5
#     }
#     return days_of_week[(day + get_code_of_month(month, year) + get_code_of_year(year)) % 7]

In [58]:
# data['day_of_week'] = data.apply(lambda x: get_day_of_week(x['day'], x['month'], x['year']), axis=1)
# data

In [59]:
# ## выходные - 6 и 7 день
# filtered_data = data[data.apply(lambda x: x['day_of_week'] in range(6, 7 + 1), axis=1)]
# filtered_data

In [60]:
# grouped_by_months = data.groupby('month')
# grouped_by_months.count()['id']